In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version
from google.colab import files
uploaded = files.upload()

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Saving housing_data.csv to housing_data.csv
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,922

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
#spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("/content/housing_data.csv"), sep=",", header=True)
homes_df.show()


+----------+---------------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7229300521|20141013T000000| 231300|       2|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000| 538000|       3|     2.25|       2570|    7242|     2|         0|   0|        3|    7|      2170|          400|    1951

In [5]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('home_sales')


In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
a = """
SELECT
  YEAR(yr_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(a).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2015|    940324.31|
|2014|    712226.14|
|2013|    729881.97|
|2012|    606583.35|
|2011|     627180.8|
|2010|    605018.37|
|2009|     553977.4|
|2008|    806330.48|
|2007|    764035.91|
|2006|    738474.06|
|2005|    667901.81|
|2004|    658634.77|
|2003|    615643.01|
|2002|    640074.65|
|2001|    731348.23|
|2000|    761055.96|
|1999|     676729.2|
|1998|     653719.7|
|1997|    624072.42|
|1996|    715369.01|
+----+-------------+
only showing top 20 rows



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
b =  """
SELECT
  YEAR(yr_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY YEAR(yr_built)
ORDER BY YEAR DESC
"""
spark.sql(b).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2015|    1550000.0|
|2014|    765652.96|
|2013|    477595.38|
|2012|     537490.0|
|2011|     645250.0|
|2010|     554125.0|
|2009|    570746.94|
|2008|    609063.64|
|2007|     610093.1|
|2006|    618956.98|
|2005|    634251.56|
|2004|    635366.67|
|2003|    724457.69|
|2002|    672418.75|
|2001|    727029.41|
|2000|    683632.86|
|1999|     766570.4|
|1998|     763375.0|
|1997|     612475.0|
|1996|    1490000.0|
+----+-------------+
only showing top 20 rows



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
c = """
SELECT
  YEAR(yr_built) AS YEAR_BUILT,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
and sqft_living >= 2000
and floors = 2
GROUP BY yr_built
ORDER BY yr_built DESC
"""
spark.sql(c).show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2015|    1550000.0|
|      2014|    933142.67|
|      2013|     570685.0|
|      2012|     572500.0|
|      2011|     639750.0|
|      2010|     721000.0|
|      2009|    1011250.0|
|      2008|    1064500.0|
|      2007|    949777.78|
|      2006|    954444.69|
|      2005|    792503.85|
|      2004|     719500.0|
|      2003|     851750.0|
|      2002|     863200.0|
|      2001|     860875.0|
|      2000|    795511.25|
|      1999|    845605.56|
|      1998|    922142.86|
|      1997|     679970.0|
|      1996|    1490000.0|
+----------+-------------+
only showing top 20 rows



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

d = """
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(d).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|   4|   1463711.24|
|   3|    971965.27|
|   2|    792400.89|
|   1|    812280.84|
|   0|    496564.67|
+----+-------------+

--- 0.9994776248931885 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

e = """
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(e).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|   4|   1463711.24|
|   3|    971965.27|
|   2|    792400.89|
|   1|    812280.84|
|   0|    496564.67|
+----+-------------+

--- 0.5703532695770264 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.partitionBy('yr_built').parquet('parquet_home_sales',mode='overwrite')

In [16]:
# 11. Read the parquet formatted data.
parquet_homes_df = spark.read.parquet('parquet_home_sales')

In [17]:
# 12. Create a temporary table for the parquet data.
parquet_homes_df.createOrReplaceTempView('parquet_temp_home')

In [18]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

e = """
SELECT
  view,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(e).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|   4|   1463711.24|
|   3|    971965.27|
|   2|    792400.89|
|   1|    812280.84|
|   0|    496564.67|
+----+-------------+

--- 0.35763120651245117 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('not cached')


not cached
